In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext nb_black
%load_ext lab_black

<IPython.core.display.Javascript object>

In [ ]:
# default_exp model

<IPython.core.display.Javascript object>

# Model

This section implements functionality concerned with generating predictions for Numerai on preprocessed data.

Currently supported frameworks and formats:
1. `.joblib` (Common format to save Python objects. These models should have a `.predict` method. Especially convenient for [sklearn models](https://scikit-learn.org/stable/supervised_learning.html).)
2. `.pickle`/`.pkl` (Arbitrary Python objects. All pickled models should have a `.predict` method.)
3. `.cbm` (Easy format to load [CatBoost](https://catboost.ai/en/docs/) models.)
4. `.lgb` (Format to load [LightGBM](https://lightgbm.readthedocs.io/en/latest/) models.)
5. `.h5` ([tf.keras](https://keras.io/) models)
6. Baseline models for which loading from files is not relevant (i.e. `ConstantModel` and `RandomModel`.)


It is recommended to use models within `ModelPipeline`s (section 6), but they can also be used on its own.

The last section of this notebook explains two different ways you can implement your own models for `numerai-blocks`:
1. From `BaseModel` (custom prediction logic).
2. From `DirectoryModel` (make predictions for all models in directory with given file suffix.
Prediction logic will already be implemented. Only write model loading logic).

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
#export
import os
import gc
import uuid
import wandb
import joblib
import pickle
import numpy as np
import pandas as pd
import lightgbm as lgb
import tensorflow as tf
from pathlib import Path
from typing import Union
from tqdm.auto import tqdm
from functools import partial
from catboost import CatBoost
from typeguard import typechecked
from abc import ABC, abstractmethod
from rich import print as rich_print
from sklearn.dummy import DummyRegressor

from numerai_blocks.download import NumeraiClassicDownloader
from numerai_blocks.numerframe import NumerFrame, create_numerframe
from numerai_blocks.preprocessing import display_processor_info

## 0. Base

### 0.1. BaseModel

The `BaseModel` is an abstract base class that handles some directory logic and naming conventions. All models should inherit from `BaseModel` and be sure to implement the `.predict` method.

In general, models are loaded in from disk. However, if no model files are involved in your model you should pass an empty string (`""`) as the `model_directory` argument.

Note that a new prediction column will have the column name `prediction_{MODEL_NAME}`.

In [ ]:
#export
class BaseModel(ABC):
    """
    Setup for model prediction on a Dataset.

    :param model_directory: Main directory from which to read in models.
    :param model_name: Name that will be used to create column names and for display purposes.
    """
    def __init__(self, model_directory: str,
                 model_name: str = None,
                 ):
        self.model_directory = Path(model_directory)
        self.model_name = model_name if model_name else uuid.uuid4().hex
        self.prediction_col_name = f"prediction_{self.model_name}"
        self.description = f"{self.__class__.__name__}: '{self.model_name}' prediction"

    @abstractmethod
    def predict(self, dataf: Union[pd.DataFrame, NumerFrame]) -> NumerFrame:
        """ Return NumerFrame with column added for prediction. """
        ...
        return NumerFrame(dataf)

    def get_prediction_col_names(self, pred_shape: tuple) -> list:
        """ Create multiple columns if predictions are multi-target. """
        prediction_cols = self.prediction_col_name
        if len(pred_shape) > 1:
            if pred_shape[1] > 1:
                prediction_cols = [f"{self.prediction_col_name}_{i}" for i in range(pred_shape[1])]
        return prediction_cols

    def __call__(self, dataf: Union[pd.DataFrame, NumerFrame]) -> NumerFrame:
        return self.predict(dataf=dataf)

### 0.2. DirectoryModel

A `DirectoryModel` assumes that you have a directory of models and you want to load + predict for all models with a certain `file_suffix` (for example, `.joblib`, `.cbm` or `.lgb`). This base class handles prediction logic for this situation.

If you are thinking of implementing your own model and this is your use case, then you should inherit from `DirectoryModel` and be sure to implement `load_models` method. Your then don't have to implement any prediction logic in the `.predict` method.

When inheriting from `DirectoryModel` the only mandatory method implementation is for `load_models`. It should instantiate all models and return them as a `list`.

In [ ]:
#export
class DirectoryModel(BaseModel):
    """
    Base class implementation where predictions are average out from a directory of models.
    Examples, JoblibModel, CatBoostModel, LGBMModel, etc.
    Walks through every file with given file_suffix in a directory.
    :param model_directory: Main directory from which to read in models.
    :param file_suffix: File format to load (For example, .joblib, .pkl, .cbm or .lgb)
    :param model_name: Name that will be used to create column names and for display purposes.
    :param feature_cols: optional list of features to use for prediction.
    Selects all feature columns (i.e. column names with prefix 'feature') by default.
    :param combine_preds: Whether to average predictions along column axis.
    Only relevant for multi target models.
    Convenient when you want to predict the main target by averaging a multi-target model.
    """
    def __init__(self, model_directory: str, file_suffix: str,
                 model_name: str = None,
                 feature_cols: list = None,
                 combine_preds = True,
                 ):
        super().__init__(model_directory=model_directory,
                         model_name=model_name,
                         )
        self.file_suffix = file_suffix
        self.model_paths = list(self.model_directory.glob(f'*.{self.file_suffix}'))
        if self.file_suffix:
            assert self.model_paths, f"No {self.file_suffix} files found in {self.model_directory}."
        self.total_models = len(self.model_paths)
        self.feature_cols = feature_cols
        self.combine_preds = combine_preds

    @display_processor_info
    def predict(self, dataf: NumerFrame, *args, **kwargs) -> NumerFrame:
        """
        Use all recognized models to make predictions and average them out.
        :param dataf: A Preprocessed DataFrame where all its features can be passed to the model predict method.
        *args, **kwargs will be parsed into the model.predict method.
        :return: A new dataset with prediction column added.
        """
        dataf.loc[:, self.prediction_col_name] = np.zeros(len(dataf))
        models = self.load_models()
        feature_cols = self.feature_cols if self.feature_cols else dataf.feature_cols
        for model in tqdm(models, desc=self.description, position=1):
            predictions = model.predict(dataf[feature_cols], *args, **kwargs)
            # Check for if model output is a Pandas DataFrame
            predictions = predictions.values if isinstance(predictions, pd.DataFrame) else predictions
            predictions = predictions.mean(axis=1) if self.combine_preds and len(predictions.shape) > 1 else predictions
            prediction_cols = self.get_prediction_col_names(predictions.shape)
            dataf.loc[:, prediction_cols] = dataf.loc[:, prediction_cols] + (predictions / self.total_models)
        del models; gc.collect()
        return NumerFrame(dataf)

    @abstractmethod
    def load_models(self) -> list:
        """ Instantiate all models detected in self.model_paths. """
        ...

## 1. Single model formats

Implementations for common Numerai model prediction situations.

### 1.1. SingleModel

In many cases you just want to load a single model file and create predictions for that model. `SingleModel` supports this.

This class supports multiple model formats for easy use. All models should have a `.predict` method.
Currently, `.joblib`, `.cbm`, `.pkl`, `.pickle` and `.h5` (keras) format are supported.

**Things to keep in mind**
- This model will use all available features in the `NumerFrame` and use them for prediction. Make sure to define proper feature selection if the models does not use all features.
- If you have XGBoost models we recommend saving them as `.joblib`.
- The added prediction column will have the column name `prediction_{MODEL_NAME}` if 1 target is predicted.
For multiple targets the new column names will be `prediction_{MODEL_NAME}_{i}` for each target number i (starting with 0).
- We welcome the Numerai community to extend `SingleModel` for more file formats. See the Contributing section in `README.md` for more information on contributing.

In [ ]:
#export
@typechecked
class SingleModel(BaseModel):
    """
    Load single model from file and perform prediction logic.
    :param model_file_path: Full path to model file.
    :param model_name: Name that will be used to create column names and for display purposes.
    :param combine_preds: Whether to average predictions along column axis.
    Only relevant for multi target models.
    Convenient when you want to predict the main target by averaging a multi-target model.
    :param autoencoder_mlp: Whether your model is an autoencoder + MLP model.
    Will take the 3rd of tuple output in this case. Only relevant for NN models.
    More info on autoencoders:
    https://forum.numer.ai/t/autoencoder-and-multitask-mlp-on-new-dataset-from-kaggle-jane-street/4338
    :param feature_cols: optional list of features to use for prediction.
    Selects all feature columns (i.e. column names with prefix 'feature') by default.
    """
    def __init__(self, model_file_path: str, model_name: str = None,
                 combine_preds = False, autoencoder_mlp = False,
                 feature_cols: list = None
                 ):
        self.model_file_path = Path(model_file_path)
        assert self.model_file_path.exists(), f"File path '{self.model_file_path}' does not exist."
        assert self.model_file_path.is_file(), f"File path must point to file. Not valid for '{self.model_file_path}'."
        super().__init__(model_directory=str(self.model_file_path.parent),
                         model_name=model_name,
                         )
        self.model_suffix = self.model_file_path.suffix
        self.suffix_to_model_mapping = {".joblib": joblib.load,
                                        ".cbm": CatBoost().load_model,
                                        ".pkl": pickle.load,
                                        ".pickle": pickle.load,
                                        ".h5": partial(tf.keras.models.load_model, compile=False)
                                        }
        self.__check_valid_suffix()
        self.combine_preds = combine_preds
        self.autoencoder_mlp = autoencoder_mlp
        self.feature_cols = feature_cols

    def predict(self, dataf: NumerFrame, *args, **kwargs) -> NumerFrame:
        model = self._load_model(*args, **kwargs)
        feature_cols = self.feature_cols if self.feature_cols else dataf.feature_cols
        predictions = model.predict(dataf[feature_cols])
        # Check for if model output is a Pandas DataFrame
        predictions = predictions.values if isinstance(predictions, pd.DataFrame) else predictions
        predictions = predictions[2] if self.autoencoder_mlp else predictions
        predictions = predictions.mean(axis=1) if self.combine_preds else predictions
        prediction_cols = self.get_prediction_col_names(predictions.shape)
        dataf.loc[:, prediction_cols] = predictions
        del model; gc.collect()
        return NumerFrame(dataf)

    def _load_model(self, *args, **kwargs):
        """ Load arbitrary model from path using suffix to model mapping. """
        return self.suffix_to_model_mapping[self.model_suffix](str(self.model_file_path), *args, **kwargs)

    def __check_valid_suffix(self):
        """ Detailed message if model is not supported in this class. """
        try:
            self.suffix_to_model_mapping[self.model_suffix]
        except KeyError:
            raise NotImplementedError(
                f"Format '{self.model_suffix}' is not available. Available versions are {list(self.suffix_to_model_mapping.keys())}"
            )


In [ ]:
dataf = create_numerframe("test_assets/mini_numerai_version_2_data.parquet")
test_paths = ["test_assets/joblib_v2_example_model.joblib"]
for path in test_paths:
    model = SingleModel(path, model_name="test")
    print(model.predict(dataf).get_prediction_data.head(2))

                  prediction_test
id                               
n559bd06a8861222         0.506948
n9d39dea58c9e3cf         0.492578


In [ ]:
model = SingleModel(test_paths[0], model_name="test")
model.suffix_to_model_mapping

{'.joblib': <function joblib.numpy_pickle.load(filename, mmap_mode=None)>,
 '.cbm': <bound method CatBoost.load_model of <catboost.core.CatBoost object at 0x7fd718206810>>,
 '.pkl': <function _pickle.load(file, *, fix_imports=True, encoding='ASCII', errors='strict')>,
 '.pickle': <function _pickle.load(file, *, fix_imports=True, encoding='ASCII', errors='strict')>,
 '.h5': functools.partial(<function load_model at 0x7fd70678b710>, compile=False)}

### 1.2. WandbKerasModel

This model is for a specific case if you are logging Keras model using [Weights & Biases](https://wandb.ai/site) and want to download the best model for a specific run. `WandbKerasModel` wraps `SingleModel` and only implements additional logic for downloading models from Weights & Biases.

To authenticate your W&B account you are given several options:
1. Run `wandb login` in terminal and follow instructions ([docs](https://docs.wandb.ai/ref/cli/wandb-login)).
2. Configure [global environment variable](https://docs.wandb.ai/guides/track/advanced/environment-variables) `"WANDB_API_KEY"`.
3. Run `wandb.init(project=PROJECT_NAME, entity=ENTITY_NAME)` and
pass API key from [https://wandb.ai/authorize](https://wandb.ai/authorize).

In [ ]:
#export
@typechecked
class WandbKerasModel(SingleModel):
    """
    Download best .h5 model from Weights & Biases (W&B) run in local directory and make predictions.
    More info on W&B: https://wandb.ai/site
    :param run_path: W&B path structured as entity/project/run_id.
    Can be copied from the Overview tab of a W&B run.
    For more info: https://docs.wandb.ai/ref/app/pages/run-page#overview-tab
    Entity, project and id can be found in Overview tab of W&B run.
    :param file_name: Name of .h5 file as saved in W&B run.
    'model-best.h5' by default.
    File name can be found under files tab of W&B run.
    :param combine_preds: Whether to average predictions along column axis.
    Convenient when you want to predict the main target by averaging a multi-target model.
    :param autoencoder_mlp: Whether your model is an autoencoder + MLP model.
    Will take the 3rd of tuple output in this case. Only relevant for NN models.
    More info on autoencoders:
    https://forum.numer.ai/t/autoencoder-and-multitask-mlp-on-new-dataset-from-kaggle-jane-street/4338
    :param replace: Replace any model files saved under the same file name with downloaden W&B run model. WARNING: Setting to True may overwrite models in your local environment.
    :param feature_cols: optional list of features to use for prediction.
    Selects all feature columns (i.e. column names with prefix 'feature') by default.

    To authenticate your W&B account you are given several options:
    1. Run wandb login in terminal and follow instructions.
    2. Configure global environment variable "WANDB_API_KEY".
    3. Run wandb.init(project=PROJECT_NAME, entity=ENTITY_NAME) and
    pass API key from https://wandb.ai/authorize
    """
    def __init__(self,
                 run_path: str,
                 file_name: str = "model-best.h5",
                 combine_preds = False,
                 autoencoder_mlp = False,
                 replace = False,
                 feature_cols: list = None
                 ):
        self.run_path = run_path
        self.file_name = file_name
        self.replace = replace

        self._download_model()
        super().__init__(model_file_path=f"{self.run_path.split('/')[-1]}_{self.file_name}",
                         model_name=self.run_path,
                         combine_preds=combine_preds,
                         autoencoder_mlp=autoencoder_mlp,
                         feature_cols=feature_cols
                         )

    def _download_model(self):
        """
        Use W&B API to download .h5 model file.
        More info on API: https://docs.wandb.ai/guides/track/public-api-guide
        """
        if Path(self.file_name).is_file() and not self.replace:
            rich_print(f":warning: [red] Model file '{self.file_name}' already exists in local environment.\
            Skipping download of W&B run model. If this is not the model you want to use for prediction\
            consider moving it or set 'replace=True' at initialization to overwrite. [/red] :warning:")
        else:
            rich_print(f":page_facing_up: [green] Downloading '{self.file_name}' from '{self.run_path}' in W&B Cloud. [/green] :page_facing_up:")
        run = wandb.Api().run(self.run_path)
        run.file(name=self.file_name).download(replace=self.replace)
        os.rename(self.file_name, f"{self.run_path.split('/')[-1]}_{self.file_name}")

In [ ]:
run_path = "user123/project/abcd1234"
# model = WandbKerasModel(run_path=run_path)

<IPython.core.display.Javascript object>

## 2. Loading all models in directory

### 2.1. Joblib directory

Many models, like `scikit-learn`, can conveniently be saved as `.joblib` files. This class automatically loads all `.joblib` files in a given folder and generates (averaged out) predictions.

In [ ]:
#export
@typechecked
class JoblibModel(DirectoryModel):
    """
    Load and predict for arbitrary models in directory saved as .joblib.
    All loaded models should have a .predict method and accept the features present in the data.
    :param model_directory: Main directory from which to read in models.
    :param model_name: Name that will be used to create column names and for display purposes.
    :param feature_cols: optional list of features to use for prediction.
    Selects all feature columns (i.e. column names with prefix 'feature') by default.
    """
    def __init__(self,
                 model_directory: str,
                 model_name: str = None,
                 feature_cols: list = None
                 ):
        file_suffix = 'joblib'
        super().__init__(model_directory=model_directory,
                         file_suffix=file_suffix,
                         model_name=model_name,
                         feature_cols=feature_cols
                         )

    def load_models(self) -> list:
        return [joblib.load(path) for path in self.model_paths]

<IPython.core.display.Javascript object>

In [ ]:
dataf = create_numerframe("test_assets/mini_numerai_version_2_data.parquet", metadata={"version": 2})
model = JoblibModel("test_assets", model_name="Joblib_LGB")
predictions = model.predict(dataf).get_prediction_data
assert predictions['prediction_Joblib_LGB'].between(0, 1).all()
predictions.head(3)

JoblibModel: 'Joblib_LGB' prediction:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Finished step DirectoryModel. Output shape=(10, 1074). Time taken for step: 
0:00:00.394503. ✅

,prediction_Joblib_LGB
id,
n559bd06a8861222,0.506948
n9d39dea58c9e3cf,0.492578
nb64f06d3a9fc9f1,0.490879


<IPython.core.display.Javascript object>

### 2.2. Catboost directory (.cbm)

This model setup loads in all `CatBoost` (`.cbm`) models present in a given directory and makes (averaged out) predictions.

In [ ]:
#export
@typechecked
class CatBoostModel(DirectoryModel):
    """
    Load and predict with all .cbm models (CatBoostRegressor) in directory.
    :param model_directory: Main directory from which to read in models.
    :param model_name: Name that will be used to define column names and for display purposes.
    :param feature_cols: optional list of features to use for prediction.
    Selects all feature columns (i.e. column names with prefix 'feature') by default.
    """
    def __init__(self,
                 model_directory: str,
                 model_name: str = None,
                 feature_cols: list = None
                 ):
        file_suffix = 'cbm'
        super().__init__(model_directory=model_directory,
                         file_suffix=file_suffix,
                         model_name=model_name,
                         feature_cols=feature_cols
                         )

    def load_models(self) -> list:
        return [CatBoost().load_model(path) for path in self.model_paths]

<IPython.core.display.Javascript object>

In [ ]:
from numerai_blocks.preprocessing import GroupStatsPreProcessor
dataf = create_numerframe("test_assets/mini_numerai_version_1_data.csv", {"version": 1})
processed_dataf = GroupStatsPreProcessor()(dataf)
model = CatBoostModel("test_assets", model_name="CB")

✅ Finished step GroupStatsPreProcessor. Output shape=(10, 332). Time taken for step: 
0:00:00.045209. ✅

<IPython.core.display.Javascript object>

In [ ]:
predictions = model.predict(processed_dataf).get_prediction_data
assert predictions['prediction_CB'].between(0, 1).all()
predictions.head(3)

CatBoostModel: 'CB' prediction:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Finished step DirectoryModel. Output shape=(10, 333). Time taken for step: 0:00:00.295325.
✅

,prediction_CB
0,0.492046
1,0.499881
2,0.485325


<IPython.core.display.Javascript object>

### 2.3. LightGBM directory (.lgb)

This model setup loads in all `LightGBM` (`.lgb`) models present in a given directory and makes (averaged out) predictions.

In [ ]:
#export
@typechecked
class LGBMModel(DirectoryModel):
    """
    Load and predict with all .lgb models (LightGBM) in directory.
    :param model_directory: Main directory from which to read in models.
    :param model_name: Name that will be used to define column names and for display purposes.
    :param feature_cols: optional list of features to use for prediction.
    Selects all feature columns (i.e. column names with prefix 'feature') by default.
    """
    def __init__(self,
                 model_directory: str,
                 model_name: str = None,
                 feature_cols: list = None
                 ):
        file_suffix = 'lgb'
        super().__init__(model_directory=model_directory,
                         file_suffix=file_suffix,
                         model_name=model_name,
                         feature_cols=feature_cols
                         )

    def load_models(self) -> list:
        return [lgb.Booster(model_file=str(path)) for path in self.model_paths]

<IPython.core.display.Javascript object>

In [ ]:
dataf = create_numerframe("test_assets/mini_numerai_version_2_data.parquet")
model = LGBMModel("test_assets", model_name="LGB")
predictions = model.predict(dataf).get_prediction_data
assert predictions['prediction_LGB'].between(0, 1).all()
predictions.head(3)

LGBMModel: 'LGB' prediction:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Finished step DirectoryModel. Output shape=(10, 1074). Time taken for step: 
0:00:00.397787. ✅

,prediction_LGB
id,
n559bd06a8861222,0.506948
n9d39dea58c9e3cf,0.492578
nb64f06d3a9fc9f1,0.490879


<IPython.core.display.Javascript object>

## 3. Baseline models

Setting a baseline is always an important step for data science problems. This section introduces models that should only be used a baselines.

### 3.1. ConstantModel

This model simply outputs a constant of your choice. Convenient for setting classification baselines.

In [ ]:
#export
class ConstantModel(BaseModel):
    """
    WARNING: Only use this Model for testing purposes.
    Create constant prediction.
    :param constant: Value for constant prediction.
    :param model_name: Name that will be used to create column names and for display purposes.
    """
    def __init__(self, constant: float = 0.5, model_name: str = None):
        self.constant = constant
        model_name = model_name if model_name else f"constant_{self.constant}"
        super().__init__(model_directory="",
                         model_name=model_name
                         )
        self.clf = DummyRegressor(strategy='constant', constant=constant).fit([0.], [0.])

    def predict(self, dataf: NumerFrame) -> NumerFrame:
        dataf.loc[:, self.prediction_col_name] = self.clf.predict(dataf.get_feature_data)
        return NumerFrame(dataf)

<IPython.core.display.Javascript object>

In [ ]:
constant = 0.85
dataf = create_numerframe("test_assets/mini_numerai_version_1_data.csv")
constant_model = ConstantModel(constant=constant)
predictions = constant_model.predict(dataf).get_prediction_data
assert (predictions.to_numpy() == constant).all()
predictions.head(3)

,prediction_constant_0.85
0,0.85
1,0.85
2,0.85


<IPython.core.display.Javascript object>

### 3.2. RandomModel

This model returns uniformly distributed predictions (range $[0...1)$). Good baseline for regression models.

In [ ]:
#export
class RandomModel(BaseModel):
    """
    WARNING: Only use this Model for testing purposes.
    Create uniformly distributed predictions.
    :param model_name: Name that will be used to create column names and for display purposes.
    """
    def __init__(self, model_name: str = None):
        model_name = model_name if model_name else "random"
        super().__init__(model_directory="",
                         model_name=model_name
                         )

    def predict(self, dataf: Union[pd.DataFrame, NumerFrame]) -> NumerFrame:
        dataf.loc[:, self.prediction_col_name] = np.random.uniform(size=len(dataf))
        return NumerFrame(dataf)

<IPython.core.display.Javascript object>

In [ ]:
dataf = create_numerframe("test_assets/mini_numerai_version_1_data.csv")
random_model = RandomModel()
predictions = random_model.predict(dataf).get_prediction_data
assert predictions['prediction_random'].between(0, 1).all()
predictions.head(3)

,prediction_random
0,0.225435
1,0.763317
2,0.296617


<IPython.core.display.Javascript object>

### 3.3. Example (validation) predictions

This Model performs downloading and adding of example predictions for Numerai Classic. Convenient when you are constructing a `ModelPipeline` (see section 6) and want to include example predictions.

In [ ]:
#export
@typechecked
class ExamplePredictionsModel(BaseModel):
    """
    Load example predictions and add to NumerFrame.
    :param file_name: File to download from NumerAPI.
    'example_validation_predictions.parquet' by default.
    :param data_directory: Directory path to download example predictions to
    or directory where example data already exists.
    :param round_num: Optional round number. Downloads most recent round by default.
    """
    def __init__(self, file_name: str = "example_validation_predictions.parquet",
                 data_directory: str = "example_predictions_model",
                 round_num: int = None):
        super().__init__(model_directory="",
                         model_name="example",
                         )
        self.file_name = file_name
        self.data_directory = data_directory
        self.round_num = round_num

    @display_processor_info
    def predict(self, dataf: NumerFrame) -> NumerFrame:
        """ Return NumerFrame with added example predictions. """
        self._download_example_preds()
        example_preds = self._load_example_preds()
        dataf.loc[:, self.prediction_col_name] = dataf.merge(example_preds, on='id', how='left')['prediction']
        self.downloader.remove_base_directory()
        return NumerFrame(dataf)

    def _download_example_preds(self):
        self.downloader = NumeraiClassicDownloader(directory_path=self.data_directory)
        self.dest_path = f"{str(self.downloader.dir)}/{self.file_name}"
        self.downloader.download_single_dataset(filename=self.file_name,
                                                dest_path=self.dest_path,
                                                round_num=self.round_num)

    def _load_example_preds(self, *args, **kwargs):
        return pd.read_parquet(self.dest_path, *args, **kwargs)

<IPython.core.display.Javascript object>

In [ ]:
# slow
# Download validation data
downloader = NumeraiClassicDownloader("example_predictions_model")
val_file = "numerai_validation_data.parquet"
val_save_path = f"{str(downloader.dir)}/{val_file}"
downloader.download_single_dataset(filename=val_file,
                                   dest_path=val_save_path)

# Load validation data and add example predictions
dataf = create_numerframe(val_save_path)
example_model = ExamplePredictionsModel()
predictions = example_model.predict(dataf).get_prediction_data
assert predictions['prediction_example'].between(0, 1).all()
predictions.head(3)

No existing directory found at 'example_predictions_model'. Creating directory...

📁 Downloading 'numerai_validation_data.parquet' 📁

2022-02-25 14:13:48,106 INFO numerapi.utils: starting download
example_predictions_model/numerai_validation_data.parquet:  12%|█▏        | 26.8M/228M [00:16<00:45, 4.41MB/s]  

KeyboardInterrupt: 

<IPython.core.display.Javascript object>

## 4. Custom Model

There are two different ways to implement new models. Both have their own conveniences and use cases.
**4.1.** Inherit from `BaseModel` (custom prediction logic).
**4.2.** Inherit from `DirectoryModel` (make predictions for all models in directory with given file suffix.
Prediction logic will already be implemented. Only implement model loading logic).

**4.1. (From BaseModel)** works well when you have no or only a single file that you use for generating predictions.
Examples:
1. Loading a model is not relevant or your model is already loaded in memory.
2. You would like predictions for one model loaded from disk.
3. The object you are loading already aggregates multiple models and transformation steps (such as [scikit-learn FeatureUnion](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.FeatureUnion.html)).

**4.2. (From DirectoryModel)** is convenient when you have a lot of similar models in a directory and want to generate predictions for all of them.
Examples:
1. You have multiple similar models saved through a cross validation process.
2. You have a bagging strategy where you have a lot of models trained on slightly different data or with different initializations.


### 4.1. From BaseModel

Arbitrary models can be instantiated and use for prediction generation by inheriting from `BaseModel`. Arbitrary logic (model loading, prediction, etc.) can be defined in `.predict` as long as the method takes a `NumerFrame` as input and outputs a `NumerFrame`. The Model should be able to typecheck by adding the `@typeguard.typechecked` decorator at the top of the class.

For clear console output we recommend adding the `@display_processor_info` decorator to the `.predict` method.

If your model does not involve reading files from disk specify `model_directory=""`.

In [ ]:
#export
@typechecked
class AwesomeModel(BaseModel):
    """
    - TEMPLATE -
    Predict with arbitrary prediction logic and model formats.

    :param model_directory: Main directory from which to read in models.
    :param model_name: Name that will be used to define column names and for display purposes.
    :param feature_cols: optional list of features to use for prediction.
    Selects all feature columns (i.e. column names with prefix 'feature') by default.
    """
    def __init__(self, model_directory: str, model_name: str = None,
                 feature_cols: list = None):
        super().__init__(model_directory=model_directory,
                         model_name=model_name,
                         )
        self.feature_cols = feature_cols

    @display_processor_info
    def predict(self, dataf: NumerFrame) -> NumerFrame:
        """ Return NumerFrame with column(s) added for prediction(s). """
        # Get all features
        feature_cols = self.feature_cols if self.feature_cols else dataf.feature_cols
        feature_df = dataf[feature_cols]
        # Predict and add to new column
        ...
        # Parse all contents of NumerFrame to the next pipeline step
        return NumerFrame(dataf)

### 4.2. From DirectoryModel

You may want to implement a setup similar to `JoblibModel` and `CatBoostModel`. Namely, load in all models of a certain type from a directory, predict for all and take the average. If this is your use case, inherit from `DirectoryModel` and be sure to implement the `load_models` method.

For a `DirectoryModel` you should specify a `file_suffix` (like `.joblib` or `.cbm`) which will be used to store all available models in `self.model_paths`.

The `.predict` method will in this case already be implemented, but can be overridden if the prediction logic is more complex. For example, if you want to apply weighted averaging or a geometric mean for models within a given directory.


Like with inheriting from `BaseModel`, This Model should also be able to typecheck by adding the `@typeguard.typechecked` decorator at the top of the class.

In [ ]:
#export
@typechecked
class AwesomeDirectoryModel(DirectoryModel):
    """
    - TEMPLATE -
    Load in all models of arbitrary file format and predict for all.

    :param model_directory: Main directory from which to read in models.
    :param model_name: Name that will be used to define column names and for display purposes.
    :param feature_cols: optional list of features to use for prediction.
    Selects all feature columns (i.e. column names with prefix 'feature') by default.
    """
    def __init__(self,
                 model_directory: str,
                 model_name: str = None,
                 feature_cols: list = None
                 ):
        file_suffix = '.anything'
        super().__init__(model_directory=model_directory,
                         file_suffix=file_suffix,
                         model_name=model_name,
                         feature_cols=feature_cols
                         )

    def load_models(self) -> list:
        """ Instantiate all models and return as a list. (abstract method) """
        ...

----------------------------------------------

In [ ]:
# hide
# Run this cell to sync all changes with library
from nbdev.export import notebook2script

notebook2script()